In [1]:
import numpy as np
import struct
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import torchvision
from torch.autograd import Variable
from torchvision import datasets, transforms
from util.util import mnist_noise
from copy import deepcopy
from scipy import spatial
import torch.cuda as cutorch

from trajectoryPlugin.plugin import API

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

cuda:0


In [2]:
"""
CNN
"""
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        #self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        #out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out
    
def accuracy(predict_y, test_y):
    score = 0
    for pred, acc in zip(predict_y, test_y):
        if pred == acc:
            score +=1
    return score / test_y.shape[0]

In [3]:
"""
MNIST DATA
"""
x_train = read_idx('data/train-images.idx3-ubyte')
y_train = read_idx('data/train-labels.idx1-ubyte')
x_test = read_idx('data/t10k-images.idx3-ubyte')
y_test = read_idx('data/t10k-labels.idx1-ubyte')
valid_idx = np.random.choice(range(60000), size=5000, replace=False)
x_valid = x_train[valid_idx]
y_valid = y_train[valid_idx]
x_train = np.delete(x_train, valid_idx, axis=0)
y_train = np.delete(y_train, valid_idx)

"""
shrink dataset to make noisy significant
"""
# subset_idx = np.random.choice(range(60000-1000), size=10000, replace=False)
# x_train = x_train[subset_idx]
# y_train = y_train[subset_idx]

"""
Add Noise label to training data
"""
y_train_noisy, noise_index = mnist_noise(y_train,0.1)

In [4]:
"""
Initializaion
"""
train_idx = np.arange(len(x_train))
x_train = np.transpose(x_train,(2,1,0))
x_valid = np.transpose(x_valid,(2,1,0))
x_test = np.transpose(x_test,(2,1,0))
x_train_tensor = torchvision.transforms.ToTensor()(x_train).unsqueeze(1)
x_valid_tensor = torchvision.transforms.ToTensor()(x_valid).unsqueeze(1)
x_test_tensor = torchvision.transforms.ToTensor()(x_test).unsqueeze(1)
y_train_tensor = torch.from_numpy(y_train.astype(np.long))
y_train_noisy_tensor = torch.from_numpy(y_train_noisy.astype(np.long))
y_valid_tensor = torch.from_numpy(y_valid.astype(np.long))
y_test_tensor = torch.from_numpy(y_test.astype(np.long))

In [5]:
"""
Here is an example of standard NN training + trajectory reweighting.
"""

# suppose there are training set and validation set, trajectory API initializaiton
### currently, our API will take care of data part in training, see below
batch_size = 100
train_dataset = Data.TensorDataset(x_train_tensor, y_train_noisy_tensor)
valid_dataset = Data.TensorDataset(x_valid_tensor, y_valid_tensor)

# we use torch dataset for initializaiton
api = API(num_cluster=3, device=device, iprint=2)
api.dataLoader(train_dataset, valid_dataset, batch_size=batch_size)

# model and its paramters
cnn = CNN()
cnn.to(device)
L2 = 0.0005
learning_rate = 0.001
num_iter = 10
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate, weight_decay=L2)

# standard training starts
epoch = 1
while epoch <= num_iter:
    print("="*20 + "epoch = {}".format(epoch) + "="*20)
    for step, (data, target, weight) in enumerate(api.train_loader): # api train_loader
        data, target, weight = data.to(device), target.to(device), weight.to(device)
        output = cnn(data)
        loss = api.loss_func(output, target, weight) # api train_loader
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # record trajectory
    api.createTrajectory(cnn)
    
    # cluster trajectory + reweight data
    if epoch > 3:
        api.clusterTrajectory() # run gmm cluster
        api.reweightData(cnn, 1e6, noise_index) # update train_loader
    
    if torch.cuda.is_available():
        print("Memory ",str(cutorch.memory_allocated(0)) , ' ', str(cutorch.max_memory_allocated(0)) , ' ' , str(cutorch.get_device_properties(0).total_memory))
    
    epoch += 1

====================epoch = 1====================
Memory  51669504   161115136   6370295808
====================epoch = 2====================
Memory  51669504   161115136   6370295808
====================epoch = 3====================
Memory  51669504   161115136   6370295808
====================epoch = 4====================
| - {0: 0, 'size': 34647, 'sim': '0.3439', 'num_special': 1, 'spe_ratio': '0.0000'}
| - {1: 1, 'size': 15105, 'sim': '0.8657', 'num_special': 279, 'spe_ratio': '0.0185'}
| - {2: 2, 'size': 5248, 'sim': '-1.7904', 'num_special': 5220, 'spe_ratio': '0.9947'}
Memory  51669504   161434624   6370295808
====================epoch = 5====================
| - {0: 0, 'size': 35703, 'sim': '0.2622', 'num_special': 1, 'spe_ratio': '0.0000'}
| - {1: 1, 'size': 14024, 'sim': '0.4545', 'num_special': 253, 'spe_ratio': '0.0180'}
| - {2: 2, 'size': 5273, 'sim': '-2.2832', 'num_special': 5246, 'spe_ratio': '0.9949'}
Memory  51669504   161434624   6370295808
====================epoch 

In [6]:
api.weight_tensor

tensor([1., 1., 1.,  ..., 1., 1., 1.])